<a href="https://colab.research.google.com/github/MathBorgess/data_science_studies/blob/main/deep_learning/recurrent/shakespeare_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://www.tensorflow.org/text/tutorials/text_generation?hl=pt-br

import tensorflow as tf
import os
import numpy as np

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [2]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [3]:
tokenize_text =  tf.keras.layers.StringLookup(mask_token=None, vocabulary=list(sorted(set(text))))
decode_tokenize_text =  tf.keras.layers.StringLookup(mask_token=None, vocabulary=list(sorted(set(text))), invert=True)
tokenize_text(['a'])

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([40])>

In [4]:
def backward_text(arr):
    return tf.strings.reduce_join(decode_tokenize_text(arr), axis=-1).numpy()

In [5]:
all_tokens = tokenize_text(tf.strings.unicode_split(text, 'UTF-8'))

tensor_set = tf.data.Dataset.from_tensor_slices(all_tokens)

In [6]:
sequence_length = 100
epoch_exemple_length = len(text) // (sequence_length + 1)

# O método batch permite converter facilmente esses caracteres individuais em sequências do tamanho desejado.

# We're gonna predict the next character using the RNN, so, to predict the word Hello, if the input is Hell, the output will be ello
dataset = tensor_set.batch(batch_size=sequence_length+1, drop_remainder=True).map(lambda sequence: [sequence[:-1], sequence[1:]])

batch_size = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
buffer_size = 10000

dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE))

vocab_size = len(tokenize_text.get_vocabulary())

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = tf.keras.models.load_model('save/shakespeare_ai')

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size,
    embedding_dim=256,
    rnn_units=1024)

'''
But for simplicity, from_logits=True means the input to crossEntropy layer is normal tensor/logits,
while if from_logits=False, means the input is a probability and usually you should have some softmax activation in your last layer.
'''
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam', loss=loss)

# # Directory where the checkpoints will be saved
# checkpoint_dir = './training_checkpoints'
# # Name of the checkpoint files
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")


# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_prefix,
#     save_weights_only=True)

model.fit(dataset, epochs=40)
model.save('save/shakespeare_ai')

OSError: ignored

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids=decode_tokenize_text, ids_from_chars=tokenize_text, temperature=1)

In [ ]:
state = None
phrase = tf.constant(['I love you, juliet'])
result = [phrase]
for i in range(100):
  phrase, state = one_step_model.generate_one_step(phrase, states=state)
  result.append(phrase)
tf.strings.join(result).numpy()